In [77]:
try:
    passes += 1
except NameError:
    %cd ..
    passes = 1

In [78]:
import pandas as pd

In [79]:
df = pd.read_csv("data/titanic_kaggle.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic), :]

In [80]:
from binarybeech.binarybeech import *
import binarybeech.utils as utils

In [81]:
import logging

logging.basicConfig(level=logging.INFO)

In [82]:
rf_titanic = RandomForest(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 4, "min_leaf_samples": 2, "min_split_samples": 2},
    sample_frac=0.3,
    n_attributes=5,
    method="classification",
)

In [83]:
rf_titanic.train(20)

A tree with 15 leafs was created
A tree with 10 leafs was created
A tree with 11 leafs was created
A tree with 11 leafs was created
A tree with 13 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 11 leafs was created
A tree with 12 leafs was created
A tree with 12 leafs was created
A tree with 11 leafs was created
A tree with 13 leafs was created
A tree with 10 leafs was created
A tree with 10 leafs was created
A tree with 14 leafs was created
A tree with 12 leafs was created
A tree with 13 leafs was created
A tree with 13 leafs was created
A tree with 11 leafs was created
A tree with 12 leafs was created


In [84]:
rf_titanic.validate_oob()

{'precision': array([0.91463415, 0.57751938]),
 'recall': array([0.77479339, 0.80978261]),
 'F-score': 0.7565671596465122,
 'accuracy': 0.7844311377245509}

In [85]:
rf_titanic.validate()

{'precision': array([0.93658537, 0.59689922]),
 'recall': array([0.78688525, 0.85555556]),
 'F-score': 0.7792151000193226,
 'accuracy': 0.8053892215568862}

In [86]:
rf_titanic.variable_importance()

{'Sex': 1.0,
 'Fare': 0.22246220302375816,
 'Age': 0.1987041036717063,
 'Pclass': 0.09287257019438439,
 'PassengerId': 0.08423326133909285,
 'SibSp': 0.06695464362850974,
 'Parch': 0.047516198704103646,
 'Embarked': 0.032397408207343464,
 'Name': 0.0,
 'Ticket': 0.0,
 'Cabin': 0.0}

In [87]:
import binarybeech.visualize as viz

viz.print_bars(rf_titanic.variable_importance(), max_width=50)

Sex       |############################### 1.0
Fare      |#######                        0.22
Age       |######                          0.2
Pclass    |###                            0.093
PassengerId|###                            0.084
SibSp     |##                             0.067
Parch     |#                              0.048
Embarked  |#                              0.032
Name      |                                0.0
Ticket    |                                0.0
Cabin     |                                0.0


In [88]:
rf_titanic.validate(df_test)

{'precision': array([0.94899818, 0.56432749]),
 'recall': array([0.77761194, 0.87330317]),
 'F-score': 0.7702059021094365,
 'accuracy': 0.8013468013468014}

In [89]:
gbt_titanic = GradientBoostedTree(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 2, "min_leaf_samples": 10, "min_split_samples": 10},
    sample_frac=0.75,
    n_attributes=9,
)

In [90]:
gbt_titanic.X_names

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [91]:
it = gbt_titanic._initial_tree()

A tree with 1 leafs was created


In [92]:
viz.print_tree(it.tree)

0.6137724550898204



In [93]:
it.tree.root

In [94]:
gbt_titanic.learning_rate = 0.01
# gbt_titanic.create_trees(200)

In [95]:
gbt_titanic.predict(df_titanic.iloc[[0]])

array([0.61377246])

In [96]:
gbt_titanic.validate(df_test)

/home/armin/git/binarybeech/binarybeech/metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(m) / np.sum(m, axis=0)


{'precision': array([0., 1.]),
 'recall': array([       nan, 0.38383838]),
 'F-score': nan,
 'accuracy': 0.3838383838383838}

#k-fold cross-validation

In [97]:
c = CART(df=df_titanic, y_name="Survived", method="classification")

In [98]:
c.train(slack=1e-3)

A tree with 107 leafs was created
A tree with 87 leafs was created
A tree with 108 leafs was created
A tree with 82 leafs was created
A tree with 90 leafs was created
A tree with 86 leafs was created


INFO:binarybeech.binarybeech:beta_best: 0.0012223002708498891


A tree with 107 leafs was created


In [99]:
c.tree.leaf_count()

31

In [100]:
viz.print_tree(c.tree)

Sex in ('female',)
├── False:Age<12.70
│   ├── False:Pclass<1.00
│   │   ├── False:Fare<48.17
│   │   │   ├── False:Fare<69.49
│   │   │   │   ├── False:0
│   │   │   │   └── True: 1
│   │   │   └── True: 0
│   │   └── True: PassengerId<564.39
│   │       ├── False:PassengerId<744.42
│   │       │   ├── False:0
│   │       │   └── True: Parch<1.94
│   │       │       ├── False:0
│   │       │       └── True: 1
│   │       └── True: Age<52.68
│   │           ├── False:0
│   │           └── True: PassengerId<390.81
│   │               ├── False:PassengerId<476.56
│   │               │   ├── False:0
│   │               │   └── True: 1
│   │               └── True: PassengerId<186.66
│   │                   ├── False:PassengerId<243.44
│   │                   │   ├── False:0
│   │                   │   └── True: 1
│   │                   └── True: 0
│   └── True: SibSp<1.87
│       ├── False:0
│       └── True: 1
└── True: Pclass<2.99
    ├── False:Fare<20.58
    │   ├── False:Embarked in 

In [101]:
c.validate(df_test)

{'precision': array([0.93624772, 0.74853801]),
 'recall': array([0.85666667, 0.87972509]),
 'F-score': 0.8517688985682996,
 'accuracy': 0.8641975308641975}